# build_new_dataset.ipynb

In [7]:
# ORIGINAL_IMAGE_DIR = '/home/aubrey/Desktop/Guam07-training-set/rawdata'
NEW_DATASET_DIR = '/home/aubrey/Desktop/Guam07-training-set/datasets/rawdatasubset'
# MODEL='/home/aubrey/Desktop/Guam07-training-set/datasets/Guam07v1/runs/detect/train5/weights/best.pt'
IMAGE_DIR_PATH = '/home/aubrey/Desktop/Guam07-training-set/rawdatasubset'
OBJECTS_DB_FILE_PATH = '/home/aubrey/Desktop/Guam07-training-set/code/rawdatasubset.sqlite3'

In [8]:
# from ultralytics import YOLO
# import numpy as np
# from glob import iglob
import os
import subprocess
import sqlite3

## Create data structure for the new dataset

In [9]:
# create destination folders
for dir in ['train', 'val', 'test']:
    os.makedirs(f'{NEW_DATASET_DIR}/{dir}')

# save data.yaml in top level folder
yaml = f'''{NEW_DATASET_DIR}
train: train
val: val
test: test
names:
  0: zero
  1: low
  2: medium
  3: high 
  4: fatal
  5: vcut'''

with open(f'{NEW_DATASET_DIR}/data.yaml', 'w') as f:
    f.write(yaml)
    
# save classes.txt in train, val and test folders
classes = f'''zero
low
medium
high
fatal
vcut'''

for dir in ['train', 'val', 'test']:
    with open(f'{NEW_DATASET_DIR}/{dir}/classes.txt', 'w') as f:
        f.write(classes)

In [10]:
conn = sqlite3.connect(OBJECTS_DB_FILE_PATH)
conn.row_factory = sqlite3.Row   # enables accessing values in results by field name

for row in conn.execute('SELECT imagepath, subset FROM detected_objects GROUP BY imagepath, subset;'):
    imagepath = row['imagepath'] 
    subset = row['subset']
    print(imagepath, subset) 
    
    # Create symlink to image
    src = imagepath
    dst = f'{NEW_DATASET_DIR}/{subset}'
    subprocess.run(['ln', '-s', src, dst])
    
    # Write labels file 
     
    s = ''
    for r in conn.execute(f'SELECT cls, x, y, w, h FROM detected_objects WHERE imagepath == "{imagepath}" ORDER BY x;'):
        s += f"{r['cls']} {r['x']:.6f} {r['y']:.6f} {r['w']:.6f} {r['h']:.6f}\n"
    s = s[:-1]   # delete final newline char
    print(s)
    
    filename = os.path.basename(imagepath).replace('.jpg', '.txt')
    filepath = f'{NEW_DATASET_DIR}/{subset}/{filename}'
    with open(filepath, 'w') as f:
        f.write(s)
    
conn.close()

/home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111715.jpg train
1 0.416908 0.956753 0.033134 0.069318
/home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111717.jpg train
1 0.331767 0.904633 0.032822 0.078079
/home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111718.jpg test
1 0.267789 0.888594 0.034105 0.068797
/home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111719.jpg train
1 0.262258 0.891938 0.044390 0.089187
/home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111720.jpg test
1 0.310081 0.879439 0.036324 0.078711
/home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111721.jpg train
2 0.399169 0.889503 0.040095 0.084331
/home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111722.jpg train
2 0.532306 0.875850 0.032717 0.077409
/home/aubrey/Desktop/Guam07-training-set/rawdatasubset/IMG_20221115_111723.jpg train
2 0.655380 0.952863 0.039334 0.087754
/home/aubrey/Deskt

## Populate train, val and test folders with symlinks to images

In [11]:
# def random_subset(subset_list, probability_list):
#     r = rng.multinomial(1, probability_list)
#     i = np.where(r == 1)[0][0]
#     return subset_list[i]
 
# rng = np.random.default_rng(42)
# subset_list = ['train', 'val', 'test']
# probability_list = [0.8, 0.1, 0.1]

# for imagepath in iglob(f'{ORIGINAL_IMAGE_DIR}/*.jpg'): 
#     destination_folder = random_subset(subset_list, probability_list)
#     src = imagepath
#     dst = f'{NEW_DATASET_DIR}/{destination_folder}'
#     subprocess.run(['ln', '-s', src, dst])

## Populate train, val and test folders with labels (*.txt files)

Reference https://stackoverflow.com/questions/76100975/yolov8-custom-save-directory-path


You can change the directory where the results are saved (save_dir) by modifying two arguments in predict: project and name
```
results = model.predict(source=xxx, save_txt = True, project="xxx", name="yyy")
```
such that:
```
save_dir=project/name
```


In [12]:
# # Perform object detection on an image using the model
# model = YOLO('/home/aubrey/Desktop/Guam07-training-set/code/runs/detect/imgsz9603/weights/best.pt')

# destination_folder = 'val'
# results = model(
#     source=f'{NEW_DATASET_DIR}/{destination_folder}/*.jpg',
#     imgsz=960,
#     save_txt=True,
#     project=NEW_DATASET_DIR,
#     name=destination_folder,
#     agnostic_nms=True,
#     iou=0.5,
#     )

# print('FINISHED')